In [204]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np
import json

In [205]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15668, 19)

In [206]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    # end要加一天
    end = pd.to_datetime(end) + pd.Timedelta(days=1)
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

In [209]:
get_post_date_range(df_posts[df_posts['from']=='weibo'],'2021-04-20','2023-12-10').shape

(2948, 19)

In [210]:
get_post_date_range(df_posts[df_posts['from']=='twitter'],'2021-04-20','2023-12-10').shape

(1445, 19)

# japan_dead_fish

In [171]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [172]:
# 查找 2023.12.1 -> 2023.12.10之间，twitter上query为japan dead fish的
df_data_twitter = df_posts[df_posts['from'] == 'twitter']
df_data_twitter = get_post_date_range(df_data_twitter,'2023-12-01','2023-12-10')
df_data_twitter = df_data_twitter[df_data_twitter['query']== "Japan dead fish"]
df_data_twitter.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_twitter['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_twitter)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [173]:
df_data_weibo = df_posts[df_posts['from'] == 'weibo']
df_data_weibo = get_post_date_range(df_data_weibo,'2023-12-01','2023-12-10')
df_data_weibo = df_data_weibo[df_data_weibo['query'] == "w9"]
df_data_weibo.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_weibo['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_weibo)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [174]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [157]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})

In [166]:
# 生成一些数据，这些数据的时间分布在

get_post_date_range(df_posts[df_posts['from'] == 'twitter'],'2021-04-20','2021-04-29','foreign_affairs_questions').shape



(26, 19)

In [167]:
get_post_date_range(df_posts[df_posts['from'] == 'twitter'],'2021-04-20','2021-04-29','japan_nuclear_wastewater').shape

(27, 19)

In [168]:
get_post_date_range(df_posts[df_posts['from'] == 'twitter'],'2021-04-20','2021-04-29','radioactive_condemn_water').shape

(28, 19)

In [178]:
ori_json = json.load(open('case1_post.json', 'r', encoding='utf-8'))

In [ ]:
ori_json

In [181]:
ppt_json = json.load(open('./json/post_PPT.json', 'r', encoding='utf-8'))

In [183]:
cluster_dict = {
    1:('Great_Wave_Kanagawa', "2021-04-20", "2021-04-29"),
    2:('foreign_affairs_questions', '2021-04-20', '2021-04-29'),
    3:('japan_nuclear_wastewater', '2021-04-20', '2021-04-29'),
    4:('radioactive_condemn_water', '2021-04-20', '2021-04-29'),
    5:('240_china_nuclear_pollution', '2023-08-21', '2023-08-30'),
    6:('70_billion_japan_water', '2023-08-21', '2023-08-30'),
    7:('cooling_water_nuclear_wastewater', '2023-08-21', '2023-08-30'),
    8:('south_korea_nuclear_discharge', '2023-08-21', '2023-09-01'),
    9:('sue_TEPCO_japan', '2023-08-21', '2023-08-30'),
    10:('radioactive_pollution_japan_sea', '2023-08-21', '2023-08-30'),
    11:('treatment_japan_waste_nuclear', '2023-08-21', '2023-08-30'),
    12: ('japan_dead_fish', '2023-12-01', '2023-12-10')
}


In [196]:
output_json = {}

In [202]:
# 取出其中的clustername
idx = 9
cluster_name = cluster_dict[idx][0]
# 把ppt_json取出
print(cluster_name)
print(len(ppt_json[cluster_name]) == len(ori_json[cluster_name]))
output_json[cluster_name] = []
for i in range(len(ppt_json[cluster_name])):
    out_item = ppt_json[cluster_name][i]
    # 替换diffusion_pattern
    # 替换is_assigned
    out_item['diffusion_pattern'] = ori_json[cluster_name][i]['diffusion_pattern']
    out_item['is_assigned'] = ori_json[cluster_name][i]['is_assigned']
    output_json[cluster_name].append(out_item)


sue_TEPCO_japan
True


In [203]:
# 保存
with open('case1_post_new.json', 'w', encoding='utf-8') as f:
    json.dump(output_json, f, ensure_ascii=False, indent=4)

In [ ]:
text = "On April 27th, when faced with Zhao Lijian's Chinese netizen painting "In Shina Tritium Surfing" posted on Twitter, Japanese Deputy Foreign Minister Masao Sato was very anxious. He said, "No matter what kind of 'Wolf Warrior Diplomacy' it is, this form is too low-level. It is unacceptable to express protests against the treatment of nuclear wastewater by polluting traditional Japanese culture. The Japanese government must also propose strict measures.""